# [Day 15](https://adventofcode.com/2024/day/15)

## Part 1

In [1]:
from typing import List, Tuple, Dict

def load_data(file:str) -> Tuple[Dict[str, List[Tuple[int, int]]], str]:
    with open(file, "r") as f:
        data = f.read().split("\n\n")
        objects = data[0].splitlines()
        moves = data[1].replace("\n", "")

        keys = ["#", "O", "@"]
        obj_dict = {}
        for y, line in enumerate(objects):
            for x, obj in enumerate(line):
                if obj in keys:
                    obj_dict.setdefault(obj,[])
                    obj_dict[obj].append((x, y))
    return obj_dict, moves

In [2]:
obj_dict, moves = load_data("data.txt")

In [3]:
def plot_positions(positions: Dict[str, List[Tuple[int, int]]], file_name: str):
    # Define grid dimensions
    width, height = sorted(positions["#"])[-1]
    grid_size = (width + 1, height + 1)

    # Initialize the grid with empty spaces
    grid = [["." for _ in range(grid_size[1])] for _ in range(grid_size[0])]

    # Place the elements in the grid
    for obj, coords in positions.items():
        for x, y in coords:
            grid[y][x] = obj

    # Write the grid to a text file
    with open(file_name, "w") as f:
        for row in grid:
            f.write("".join(row) + "\n")

In [4]:
plot_positions(obj_dict, "initial_grid.txt")

In [8]:
from time import sleep
from copy import deepcopy

def add_position(p1: tuple, p2: tuple) -> tuple:
    return (p1[0] + p2[0], p1[1] + p2[1])

def push(pos: tuple, step: tuple, positions: Dict[str, List[Tuple[int, int]]]):
    """
    Recursively pushes O's in the direction of step, stopping if a block (# or boundary)
    is encountered or no more O's can move.
    """
    next_pos = add_position(pos, step)

    # Check if the next position is blocked
    if next_pos in positions['#']:
        return False # Stop pushing, blocked

    # Recursively push the next O, if it exists
    if next_pos in positions['O']:
        return push(next_pos, step, positions)
    return True  # Push was successful

def update(current_pos, step, positions):

    next_pos = add_position(current_pos, step)

    if next_pos in positions['O']:
        positions["O"].remove(current_pos)
        positions["O"].append(next_pos)
        return update(next_pos, step, positions)
    else:
        positions["O"].remove(current_pos)
        positions["O"].append(next_pos)

def evolution(obj_dict: Dict[str, List[Tuple[int, int]]], moves: str):
    positions = deepcopy(obj_dict) # make a copy
    move_dict = {"<": (-1, 0), ">": (1, 0), "^": (0, -1), "v": (0, 1)}

    for move in moves:
        #sleep(1)
        #plot_positions(positions, "grid.txt")
        step = move_dict[move]  # get direction/step
        current_pos = positions["@"][0]  # get robot position
        new_pos = add_position(current_pos, step)

        # Check if new position is valid
        if new_pos in positions['#']:  # Blocked by an unmovable object
            continue

        # Main movement logic for @
        if new_pos in positions['O']: # @ tries to push an O
            if push(new_pos, step, positions): # If push is possible
                positions["@"].remove(current_pos)
                positions["@"].append(new_pos)
                update(new_pos, step, positions)
                    
        else:  # If no O is encountered, move @ normally
            positions['@'].remove(current_pos)
            positions['@'].append(new_pos)

    return positions

positions = evolution(obj_dict, moves)

In [9]:
plot_positions(positions, "final_grid.txt")

In [10]:
gps_coordinate = 0
for x, y in positions['O']:
    gps_coordinate += 100 * y + x
print(gps_coordinate)

1478649
